# This implementation of the genetic algorithm for the shortfalls has the exact seed used for MATER's requriements

This is meant to demonstrate replication and is not the way the this seed was found

Must be run on: python 3.12.0, SciPy 1.13.1, and numpy 2.0.1

In [1]:
%reset
import numpy as np
import csv
from scipy.optimize import *
import sys
import matplotlib.pyplot as plt
np.set_printoptions(threshold=sys.maxsize)
import datetime

# Loads in the analog payloads and the shortfall scores

In [2]:
#import cell

#tech
tech_array = []

#take data from tech csv and convert it to an array
with open('Found_Technologies.csv', 'rt') as f:
    array_x = csv.reader(f, skipinitialspace=True, quotechar="'")
    for line in array_x:
        tech_array.append(line)

#take data from scores tech csv and convert it to an array
tech_score = []
with open('Shortfalls_scores.csv', 'rt') as f:
    array_x = csv.reader(f, skipinitialspace=True, quotechar="'")
    for line in array_x:
        tech_score.append(line)

#print(tech_score)


# Prepares data from the .csv files and sets parameter values

In [4]:
#data processing cell

#Parse shortfall data 
tech_array_mod          = tech_array[1:]                                    #cut off title score
tech_mass_array         = np.asarray([i[-1] for i in tech_array_mod],float)      #Masses of techs 0 indexed
tech_power_array        = np.asarray([i[-2] for i in tech_array_mod],float)      #power consumption of techs 0 indexed
shortfall_array_temp    = [np.array(i[1:-2]) for i in tech_array_mod]       
shortfall_array         = []
for i in shortfall_array_temp:
    arr = []
    for j in i:
        s = j
        y = ""
        for char in s:
            if char.isdigit():
                y += char
        num = int(y)
        arr.append(num)
    
    shortfall_array.append(arr)

shortfall           = np.asarray(tech_score[1:],float)
shortfall_matrix    = np.zeros((len(shortfall_array),len(shortfall)))
for i in range(len(shortfall_matrix)):
    for j in shortfall_array[i]:
        shortfall_matrix[i][j-1] = 1

shortfall_matrix = np.transpose(shortfall_matrix) #matrix of shortfalls where the column are a tech and the value (0 or 1) represents of that index of a shortfall is completed

shortfall_integrated    = shortfall[0:, 1]  #Scores from NASA 0 indexed
shortfall_consequence   = shortfall[0:, 2]  #Scores from consequence axis 0 indexed
shortfall_liklihood     = shortfall[0:, 3]  #Scores from liklihood axis 0 indexed


beta = 185 #kg/kW Power to mass ratio
max_mass    = 40000 #kg
min_mass    = 15000 #kg
mass_ratio  =  0.3 #kg

# The score function being optimized

In [5]:
#score function for tech

def score_tech(mission, check_mass=True):
    #weights
    w1 = 2.3  #completed_score
    w2 = 7  #integrated_score
    w3 = 4  #consequence_score
    w4 = 4  #liklihood_score

    mass_max_tech = max_mass*(1-mass_ratio) #in kg
    mass_min_tech = min_mass*(1-mass_ratio) #in kg

    #Mass check
    if check_mass and (tech_mass_array.dot(mission) + beta*tech_power_array.dot(mission) <= mass_min_tech or tech_mass_array.dot(mission) + beta*tech_power_array.dot(mission) > mass_max_tech):
        return abs(tech_mass_array.dot(mission) + beta*tech_power_array.dot(mission))

    #normlizers
    max_number      = 93        #normalize number score 0-1 
    max_integrated  = 8.1035    #normalize integrated score 0-1
    max_consequence = 5         #normlize consquence score 0-1
    max_liklihood   = 5         #normlize consquence score 0-1  

    #normlize weights to sum to 10
    W  = w1+w2+w3+w4 
    w1 = 10*w1/W
    w2 = 10*w2/W
    w3 = 10*w3/W
    w4 = 10*w4/W

    mission                     = mission.astype(int) #Convete to ints
    completed                   = np.clip(shortfall_matrix @ mission, a_min=None, a_max=1) #A shortfall is either completed or not
    completed_number_score      = np.linalg.norm(completed)**2                             #Number of shortfalls completed
    normalized_completed        = completed/completed_number_score                         #Normlizes completed array to weight each score

    completed_integrated_score  = shortfall_integrated  @ normalized_completed             #Generates integrated score
    completed_consequence_score = shortfall_consequence @ normalized_completed             #Generates the consquences score
    completed_liklihood_score   = shortfall_liklihood   @ normalized_completed             #Generates the liklihood score

    #score calculation
    score = w1*completed_number_score/max_number + w2*completed_integrated_score/max_integrated + w3*completed_consequence_score/max_consequence + w4*completed_liklihood_score/max_liklihood
    return -score


# Initial Guess

In [6]:
scores_per_tech = np.zeros(len(tech_array)-1)
for i in range(len(scores_per_tech)):
    input = np.zeros(len(tech_array)-1)
    input[i] = 1
    scores_per_tech[i] = score_tech(input,check_mass=False)
indcies_tech = np.argsort(scores_per_tech)
scores_per_tech_sort = np.sort(scores_per_tech) 

#rint(scores_per_tech_sort)
#print(indcies_tech)
mass = 0
bring_array_tech = np.zeros(len(tech_array)-1)
for i in range(len(indcies_tech)):
    if mass < 0.5*(max_mass+min_mass)*(1-mass_ratio):
        bring_array_tech[indcies_tech[i]] = 1
        mass = tech_mass_array.dot(bring_array_tech) + beta*tech_power_array.dot(bring_array_tech)

print(bring_array_tech)
print(score_tech(bring_array_tech))
# print(mass)
    

[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0.
 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
-8.051954485926398


# Differential Evolution call

In [7]:
#Optimizer for tech https://en.wikipedia.org/wiki/Differential_evolution https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html#scipy.optimize.differential_evolution
bounds_tech = Bounds(lb=np.zeros(len(shortfall_array)), ub=np.ones(len(shortfall_array)))     #we either bring or we dont
seed = 900937 #np.random.randint(9999999)
res_tech    = differential_evolution(func=score_tech, bounds=bounds_tech, x0=bring_array_tech,integrality=np.ones(len(tech_array)-1),updating='deferred',tol=0.01,init='sobol',popsize=16,recombination=0.05,mutation=(0.7,1.7),maxiter=100000,seed=seed)
print(res_tech)
print('seed: ' + str(seed))

             message: Optimization terminated successfully.
             success: True
                 fun: -8.059745523995822
                   x: [ 1.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
                 nit: 290
                nfev: 595968
          population: [[ 1.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
                       [ 1.000e+00  1.000e+00 ...  0.000e+00  0.000e+00]
                       ...
                       [ 1.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
                       [ 0.000e+00  1.000e+00 ...  0.000e+00  0.000e+00]]
 population_energies: [-8.060e+00 -7.916e+00 ... -8.023e+00 -8.017e+00]
seed: 900937


# Presents and saves outputs

In [8]:
#Presenter for tech

x_tech = res_tech['x']
x_tech = x_tech.astype(int)
techs_brought = []
j = 0
mass_tech = 0
power_tech = 0
for i in range(len(x_tech)):
    if x_tech[i] == 1:
        techs_brought.append(tech_array_mod[i])
        mass_tech += tech_mass_array[i]
        power_tech += tech_power_array[i]
        j += 1
mass_tech   = round(mass_tech, 3)
power_tech  = round(power_tech, 3)
tech_score = round(-score_tech(x_tech), 3)
print("For tech demos:")
print()
print(" Number of Payloads              :   " + str(len(techs_brought)))
print(" Mass of Mission                 :   " + str(mass_tech))
print(" Power Consumption of Mission    :   " + str(power_tech))
print(" Combined Mass of Mission        :   "  + str(mass_tech + beta*power_tech))
print(" Score of Mission                :   " + str(tech_score))
print()
print(" Payloads:")
for i in techs_brought:
    print("     " +i[0])

now = datetime.datetime.now()
formatted_time = now.strftime("%d-%m-%Y_%H_%M_%S")
#write to csv
with open("raw_shortfall_out_" + formatted_time + '.csv', 'w', newline='') as save:
    wr = csv.writer(save)
    wr.writerow(["", "Name", "Shortfalls", "Power", "Mass", "Total Mass"])
    for i in techs_brought:
        arr = []
        arr.append("")
        arr.append(i[0])
        arr.append(i[1:-2])
        arr.append(i[-2])
        arr.append(i[-1])
        wr.writerow(arr)
    wr.writerow(["Totals", len(techs_brought), tech_score, power_tech, mass_tech, mass_tech + beta*power_tech, 'seed:', seed])
    

For tech demos:

 Number of Payloads              :   32
 Mass of Mission                 :   15814.967
 Power Consumption of Mission    :   64.946
 Combined Mass of Mission        :   27829.977
 Score of Mission                :   8.06

 Payloads:
     Remote Agent Architecture
     Teleoperated Rovers
     Virtual Camera
     Combustion Integrated Rack (CIR)
     perseverance Rover sensors
     ATHLETE Rover
     Extravehicular mobility unit
     Electrodynamic dust shield (EDS)
     Rover docking port
     Mars Rover (Opportunity)
     Wind Sensor
     Rad Counter measures
     Rad monitoring
     rover payloads
     Water management
     Prediction Modeling of Cryogenic Fluid Dynamics and Operations
     Dust mitigation
     Batteries
     MAV Prop
     Extraction and separation of oxygen from extraterrestrial materials
     Health monitoring for systems
     Cryogenic liquification
     A drill
     Big MOXIE
     Sensor with NPC and APC
     Small Fission Power
     3D EDL Sensor